<h1>Exercise 46</h1>
<p></p>
<strong>Expected outcome:</strong>
<br>
<img src="Day-46-Expected_output.png" width="300">

In [1]:
import os
import sqlite3
import pandas as pd

db_name="exercise46.db"
conn=sqlite3.connect(db_name)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS drivers(id VARCHAR(10), start_time TIME, end_time TIME, start_loc VARCHAR(10), end_loc VARCHAR(10))""")
conn.commit()

drivers_data=[
    ('dri_1', '09:00', '09:30', 'a','b'),
    ('dri_1', '09:30', '10:30', 'b','c'),
    ('dri_1','11:00','11:30', 'd','e'),
    ('dri_1', '12:00', '12:30', 'f','g'),
    ('dri_1', '13:30', '14:30', 'c','h'),
    ('dri_2', '12:15', '12:30', 'f','g'),
    ('dri_2', '13:30', '14:30', 'c','h')
]
cursor.executemany("INSERT INTO drivers VALUES(?,?,?,?,?)",drivers_data)
conn.commit()

querytoexec="SELECT * FROM drivers"
df=pd.read_sql(querytoexec,conn)
print(df)

      id start_time end_time start_loc end_loc
0  dri_1      09:00    09:30         a       b
1  dri_1      09:30    10:30         b       c
2  dri_1      11:00    11:30         d       e
3  dri_1      12:00    12:30         f       g
4  dri_1      13:30    14:30         c       h
5  dri_2      12:15    12:30         f       g
6  dri_2      13:30    14:30         c       h


In [2]:
firstsolution="SELECT id, COUNT(1) AS total_rides, SUM(CASE WHEN end_loc=next_start_location THEN 1 ELSE 0 END) AS profit_rides FROM (SELECT *, LEAD(start_loc,1) OVER(PARTITION BY id ORDER BY start_time ASC) AS next_start_location FROM drivers) as A GROUP BY id"
df=pd.read_sql(firstsolution,conn)
print(df)

      id  total_rides  profit_rides
0  dri_1            5             1
1  dri_2            2             0


In [3]:
secondsol="WITH rides AS (SELECT *, row_number() OVER (PARTITION BY id ORDER BY start_time ASC) AS rn FROM drivers) SELECT r1.id, COUNT(1) AS total_rides, COUNT(r2.id) AS profit_rides FROM rides AS r1 LEFT JOIN rides AS r2 ON r1.id=r2.id AND r1.end_loc=r1.start_loc AND r1.rn+1=r2.rn GROUP BY r1.id"
df=pd.read_sql(secondsol,conn)
print(df)
conn.close()

      id  total_rides  profit_rides
0  dri_1            5             0
1  dri_2            2             0
